# How to train the Baseline Models for the SENSORIUM track

### This notebook will show how to
- instantiate dataloader for the Sensorium track
- instantiate pytorch model
- instantiate a trainer function
- train two baselines for this competition track
- save the model weights (the model weights can already be found in './model_checkpoints/pretrained/')

### Imports

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

### Instantiate DataLoader

In [2]:
# loading the SENSORIUM dataset
filenames = ['../data/static26872-17-20-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': False,
                 'batch_size': 128,
                 'scale':0.25,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

# Instantiate State of the Art Model (SOTA)

In [3]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': False,
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

## Configure Trainer

In [6]:
trainer_fn = "sensorium.training.standard_trainer"

trainer_config = {'max_iter': 200,
                 'verbose': False,
                 'lr_decay_steps': 4,
                 'avg_loss': False,
                 'lr_init': 0.009,
                 }

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

# Run model training

In [7]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

Epoch 3:  34%|███▍      | 12/35 [00:01<00:02, 11.23it/s]


KeyboardInterrupt: 

### Save model checkpoints after training is complete

In [6]:
torch.save(model.state_dict(), './model_checkpoints/sensorium_sota_model.pth')

## Load Model Checkpoints

In [7]:
new_def = torch.load('./model_checkpoints/sensorium_sota_model.pth')

In [8]:
old_def = torch.load('./model_checkpoints/pretrained/sensorium_sota_model.pth')

In [9]:
set(list(new_def.keys())).difference(set(list(old_def.keys())))

{'core.features.layer1.ds_conv.in_depth_conv.bias',
 'core.features.layer1.ds_conv.out_depth_conv.bias',
 'core.features.layer1.ds_conv.spatial_conv.bias',
 'core.features.layer2.ds_conv.in_depth_conv.bias',
 'core.features.layer2.ds_conv.out_depth_conv.bias',
 'core.features.layer2.ds_conv.spatial_conv.bias',
 'core.features.layer3.ds_conv.in_depth_conv.bias',
 'core.features.layer3.ds_conv.out_depth_conv.bias',
 'core.features.layer3.ds_conv.spatial_conv.bias'}

In [10]:
set(list(old_def.keys())).difference(set(list(new_def.keys())))

set()

In [11]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_sota_model.pth"));

RuntimeError: Error(s) in loading state_dict for FiringRateEncoder:
	Missing key(s) in state_dict: "core.features.layer1.ds_conv.in_depth_conv.bias", "core.features.layer1.ds_conv.spatial_conv.bias", "core.features.layer1.ds_conv.out_depth_conv.bias", "core.features.layer2.ds_conv.in_depth_conv.bias", "core.features.layer2.ds_conv.spatial_conv.bias", "core.features.layer2.ds_conv.out_depth_conv.bias", "core.features.layer3.ds_conv.in_depth_conv.bias", "core.features.layer3.ds_conv.spatial_conv.bias", "core.features.layer3.ds_conv.out_depth_conv.bias". 

In [12]:
model.load_state_dict(torch.load("./model_checkpoints/sensorium_sota_model.pth"));

In [13]:
validation_score

0.29481468

---

# Train a simple LN model

Our LN model has the same architecture as our CNN model (a convolutional core followed by a gaussian readout)
but with all non-linearities removed except the final ELU+1 nonlinearity.
Thus turning the CNN model effectively into a fully linear model followed by a single output non-linearity.


In [14]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 3,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'grid_mean_predictor': {'type': 'cortex',
              'input_dimensions': 2,
              'hidden_layers': 1,
              'hidden_features': 30,
              'final_tanh': True},
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
              'linear': True
               }
model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

In [15]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

Epoch 110: 100%|██████████| 35/35 [00:03<00:00,  9.72it/s]


In [16]:
torch.save(model.state_dict(), './model_checkpoints/sensorium_ln_model.pth')

In [17]:
validation_score

0.20398675

In [ ]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_ln_model.pth"));

---